This file will extract the Point Cloud data from a rosbag file, and save the Point Cloud data of each timestamp as a .csv file.

The file will create a folder to save the point cloud data.

The .csv file will be named by frame (timestamp) numbers.

In [1]:
import rosbag2_py
import pandas as pd
from sensor_msgs.msg import PointCloud2
import os
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message
import numpy as np

In [2]:
# Bag file path and filename, Change it when you need to
bag_file_path = '/home/yz27246-admin/test_bag'

# Point Cloud topic
CAM1_PC = '/cam_1/zed_node/point_cloud/cloud_registered'

In [3]:
# Define and initialize ROS2 bag reader
reader = rosbag2_py.SequentialReader()

storage_options = rosbag2_py.StorageOptions(uri=bag_file_path)

converter_options = rosbag2_py.ConverterOptions(
        input_serialization_format='cdr', output_serialization_format='cdr')

reader.open(storage_options, converter_options)

[INFO] [1707262320.773693807] [rosbag2_storage]: Opened database '/home/yz27246-admin/test_bag/test_bag_0.db3' for READ_ONLY.


In [4]:
import os
# Create directory and folder for saving CSV files
point_cloud_dir = 'point_clouds'
os.makedirs(point_cloud_dir, exist_ok=True)

In [5]:
# Convert message type
msg_type = get_message('sensor_msgs/msg/PointCloud2')

In [6]:
# Function to convert point cloud data to a pandas DataFrame
def point_cloud2_to_df(msg):
    # Import here to handle the dynamic nature of PointCloud2
    from sensor_msgs_py import point_cloud2
    # convert point cloud to numpy array
    cloud_points = np.array(point_cloud2.read_points(msg, skip_nans=False))
    # convert the data to Dataframe
    df = pd.DataFrame(cloud_points)
    return df

In [7]:
while reader.has_next():
    # read the data at the next timestamp
    (topic_name, data, timestamp) = reader.read_next()
    if topic_name == CAM1_PC:
        pc_msg = deserialize_message(data, msg_type)
        df = point_cloud2_to_df(pc_msg)
        csv_filename = f"{point_cloud_dir}/point_cloud_{timestamp}.csv"
        df.to_csv(csv_filename, index=False)